# ADS 509 Module 1 Part 1: Pulling Twitter Descriptions 

This notebook holds part one of the assignment for Module 1 in ADS 509, Applied Text Mining. Work through this notebook, writing code, and answering questions where required. 

For this assignment you have chosen two musical artists who have at least 100,000 Twitter followers and 20 songs with lyrics on AZLyrics.com. In this part of the assignment we pull the some of the user information for the followers of your artist and store them in text files. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [ ]:
import tweepy
import os
import datetime
from pprint import pprint


In [ ]:
# Use this cell for any import statements you add



We need bring in our API keys. Since API keys should be kept secret, we'll keep them in a file called `api_keys.py`. This file should be stored in the directory where you store this notebook. The example file is provided for you on Blackboard. The example has API keys that are _not_ functional, so you'll need to get Twitter credentials and replace the placeholder keys. 

In [ ]:
from api_keys import api_key, api_key_secret, access_token, access_token_secret 

In [ ]:
auth = tweepy.AppAuthHandler(api_key, api_key_secret)
api = tweepy.API(auth,wait_on_rate_limit=True,wait_on_rate_limit_notify=True)

print ('API Host', api.host)
print ('API Version', api.api_root)

## Testing the API

The Twitter APIs are quite rich. Let's play around with some of the features before we dive into this section of the assignment. For our testing, it's convenient to have a small data set to play with. We will seed the code with the handle of John Chandler, one of the instructors in this course. His handle is `@37chandler`. Feel free to use a different handle if you would like to look at someone else's data. 

We will write code to explore a few aspects of the API: 

1. Pull all the follower IDs for @katymck.
1. Explore the user object, which gives us information about Twitter users. 
1. Pull some user objects for the followers. 
1. Pull the last few tweets by @katymck.


In [ ]:
handle = "37chandler"

followers = []

for page in tweepy.Cursor(api.followers_ids,
                          # This is how we will get around the issue of not being able to grab all ids at once
                          # Once the rate limit is hit, we will be  that we must wait 15 mins (900 secs)
                          wait_on_rate_limit=True, 
                          wait_on_rate_limit_notify=True, 
                          compression=True,
                          screen_name=handle).pages():

    # The page variable comes back as a list, so we have to use .extend rather than .append
    followers.extend(page)
        
        
print(f"Here are the first five follower ids for {handle} out of the {len(followers)} total.")
followers[:5]

We have the follower IDs, which are unique numbers identifying the user, but we'd like to get some more information on these users. Twitter allows us to pull "fully hydrated user objects", which is a fancy way of saying "all the information about the user". Let's look at user object for our starting handle.

In [ ]:
user = api.get_user(handle) 
pprint(user._json)

Now a few questions for you about the user object.

Q: How many fields are being returned in the \_json portion of the user object? 

A: <!-- Put your answer here --> 

---

Q: Are any of the fields within the user object non-scaler? TK correct term

A: <!-- Put your answer here --> 

---

Q: How many friends, followers, favorites, and statuses does this user have? 

A: <!-- Put your answer here --> 


We can map the follower IDs onto screen names by accessing the screen_name key within the user object. Modify the code below to also print out how many people the follower is following and how many followers they have. 

In [ ]:
ids_to_lookup = followers[:10]

for user_obj in api.lookup_users(ids_to_lookup) :
    print(f"{handle} is followed by {user_obj.screen_name}")
    
    # Add code here to print out friends and followers of `handle`


Although you won't need it for this assignment, individual tweets (called "statuses" in the API) can be a rich source of text-based data. To illustrate the concepts, let's look at the last few tweets for this user. You are encouraged to explore the `status` object and marvel in the richness of the data that is available. 


In [ ]:
tweet_count = 0

for status in tweepy.Cursor(api.user_timeline, id=handle).items():
    tweet_count += 1
    
    print(f"The tweet was tweeted at {status.created_at}.")
    print(f"The original tweet has been retweeted {status.retweet_count} times.")
    
    clean_status = status.text
    clean_status = clean_status.replace("\n"," ")
    
    print(f"{clean_status}")
    print("\n"*2)
        
    if tweet_count > 10 :
        break



## Pulling Follower Information

In this next section of the assignment, we will pull information about the followers of your two artists. We must first get the follower IDs, then we will be able to "hydrate" the IDs, pulling the user objects for them. Once we have those user objects we will extract some fields that we can use in future analyses. 


The Twitter API only allows users to make 15 requests per 15 minutes when pulling followers. Each request allows you to gather 5000 follower ids. Tweepy will grab the 15 requests quickly then wait 15 minutes, rather than slowly pull the requests over the time period. Before we start grabbing follower IDs, let's first just check how long it would take to pull all of the followers. To do this we use the `followers_count` item from the user object. 

In [ ]:
# I'm putting the handles in a list to iterate through below
handles = ['robynkonichiwa','cher']

# This will iterate through each Twitter handle that we're collecting from
for screen_name in handles:
    
    # Tells Tweepy we want information on the handle we're collecting from
    # The next line specifies which information we want, which in this case is the number of followers 
    user = api.get_user(screen_name) 
    followers_count = user.followers_count

    # Let's see roughly how long it will take to grab all the follower IDs. 
    print(f'''
    @{screen_name} has {followers_count} followers. 
    That will take roughly {followers_count/(5000*15*4):.2f} hours to pull the followers.
    ''')
    

As we pull data for each artist we will write their data to a folder called "twitter", so we will make that folder if needed.

In [ ]:
# Make the "twitter" folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then "unlink" it. Then create a new one.

if not os.path.isdir("twitter") : 
    #shutil.rmtree("twitter/")
    os.mkdir("twitter")

In this following cells, use the `api.followers_ids` (and the `tweepy.Cursor` functionality) to pull some of the followers for your two artists. As you pull the data, write the follower ids to a file called `[artist name]_followers.txt` in the "twitter" folder. For instance, for Cher I would create a file named `cher_followers.txt`. As you pull the data, also store it in an object like a list or a data frame.

In [ ]:
num_followers_to_pull = 60*1000 # feel free to use this to limit the number of followers you pull.

In [ ]:
# Modify the below code stub to pull the follower IDs and write them to a file. 

# Grabs the time when we start making requests to the API
start_time = datetime.datetime.now()

for handle in handles :
    
    output_file = handle + "_followers.txt"
    
    
    # Pull and store the follower IDs
    
    
    # Write the IDs to the output file in the `twitter` folder.
        
        
    # If you've pulled num_followers_to_pull, feel free to break out paged twitter API response
            
        
        
# Let's see how long it took to grab all follower IDs
end_time = datetime.datetime.now()
print(end_time - start_time)



Now that you have your follower ids, gather some information that we can use in future assignments on them. Using the `lookup_users` function, pull the user objects for your followers. These requests are limited to 900 per 15 minutes, but you can request 100 users at a time. At 90,000 users per 15 minutes, the rate limiter on pulls might be bandwidth rather than API limits. 

Extract the following fields from the user object: 

* screen_name	
* name	
* id	
* location	
* followers_count	
* friends_count	
* description

These can all be accessed via these names in the object. Store the fields with one user per row in a tab-delimited text file with the name `[artist name]_follower_data.txt`. For instance, for Cher I would create a file named `cher_follower_data.txt`. 


The user's description can have tabs or returns in it, so make sure to clean those out of the description before writing them to the file. 

In [ ]:

# in this cell, do the following
# 1. Set up a data frame or dictionary to hold the user information
# 2. Use the `lookup_users` api function to pull sets of 100 users at a time
# 3. Store the listed fields in your data frame or dictionary.
# 4. Write the user information in tab-delimited form to the follower data text file. 
